<a href="https://colab.research.google.com/github/getarnt/Customer-Churn-Prediction/blob/main/Telecom_Churn_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('telecom-churn-model')
jovian.set_colab_id('1N94rViIdSSbV6Lu-Azlqy4znkjNzleg6')

     |████████████████████████████████| 68 kB 2.9 MB/s 


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve

In [3]:
pd.set_option("display.max_columns", 300) 
pd.set_option("display.max_rows", 300)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
telco = pd.read_csv('/content/drive/MyDrive/Telecom case Study Asish shared/telecom_churn_data.csv')

In [4]:
#telco_raw=pd.read_csv('F:/iiit b/Python/Telecom Churn Model/telecom_churn_data.csv')

FileNotFoundError: ignored

In [9]:
telco=telco_raw

NameError: ignored

### Handling Missing Values

In [10]:
tf=telco.isna().sum()/len(telco)*100

In [11]:
k=tf[tf>50].index

As there are no values for date of last recharge or the other data columns we can assume that the value is zero and impute with zero except for night pack

Dropping Night Pack as the columns have high missing value and they do not have a pattern

In [12]:
tf[(tf>6) & (tf<10)]

onnet_mou_9         7.745077
offnet_mou_9        7.745077
roam_ic_mou_9       7.745077
roam_og_mou_9       7.745077
loc_og_t2t_mou_9    7.745077
loc_og_t2m_mou_9    7.745077
loc_og_t2f_mou_9    7.745077
loc_og_t2c_mou_9    7.745077
loc_og_mou_9        7.745077
std_og_t2t_mou_9    7.745077
std_og_t2m_mou_9    7.745077
std_og_t2f_mou_9    7.745077
std_og_t2c_mou_9    7.745077
std_og_mou_9        7.745077
isd_og_mou_9        7.745077
spl_og_mou_9        7.745077
og_others_9         7.745077
loc_ic_t2t_mou_9    7.745077
loc_ic_t2m_mou_9    7.745077
loc_ic_t2f_mou_9    7.745077
loc_ic_mou_9        7.745077
std_ic_t2t_mou_9    7.745077
std_ic_t2m_mou_9    7.745077
std_ic_t2f_mou_9    7.745077
std_ic_t2o_mou_9    7.745077
std_ic_mou_9        7.745077
spl_ic_mou_9        7.745077
isd_ic_mou_9        7.745077
ic_others_9         7.745077
dtype: float64

All the above columns are missing together which mean it can be replaced with zero

In [13]:
k1=tf[(tf>6) & (tf<10)].index

In [14]:
tf[(tf>4) & (tf<6)]

onnet_mou_8            5.378054
offnet_mou_8           5.378054
roam_ic_mou_8          5.378054
roam_og_mou_8          5.378054
loc_og_t2t_mou_8       5.378054
loc_og_t2m_mou_8       5.378054
loc_og_t2f_mou_8       5.378054
loc_og_t2c_mou_8       5.378054
loc_og_mou_8           5.378054
std_og_t2t_mou_8       5.378054
std_og_t2m_mou_8       5.378054
std_og_t2f_mou_8       5.378054
std_og_t2c_mou_8       5.378054
std_og_mou_8           5.378054
isd_og_mou_8           5.378054
spl_og_mou_8           5.378054
og_others_8            5.378054
loc_ic_t2t_mou_8       5.378054
loc_ic_t2m_mou_8       5.378054
loc_ic_t2f_mou_8       5.378054
loc_ic_mou_8           5.378054
std_ic_t2t_mou_8       5.378054
std_ic_t2m_mou_8       5.378054
std_ic_t2f_mou_8       5.378054
std_ic_t2o_mou_8       5.378054
std_ic_mou_8           5.378054
spl_ic_mou_8           5.378054
isd_ic_mou_8           5.378054
ic_others_8            5.378054
date_of_last_rech_9    4.760048
dtype: float64

In [15]:
k2=tf[(tf>4) & (tf<6)].index

In [16]:
tf[(tf>2) & (tf<4)]

onnet_mou_6            3.937039
onnet_mou_7            3.859039
offnet_mou_6           3.937039
offnet_mou_7           3.859039
roam_ic_mou_6          3.937039
roam_ic_mou_7          3.859039
roam_og_mou_6          3.937039
roam_og_mou_7          3.859039
loc_og_t2t_mou_6       3.937039
loc_og_t2t_mou_7       3.859039
loc_og_t2m_mou_6       3.937039
loc_og_t2m_mou_7       3.859039
loc_og_t2f_mou_6       3.937039
loc_og_t2f_mou_7       3.859039
loc_og_t2c_mou_6       3.937039
loc_og_t2c_mou_7       3.859039
loc_og_mou_6           3.937039
loc_og_mou_7           3.859039
std_og_t2t_mou_6       3.937039
std_og_t2t_mou_7       3.859039
std_og_t2m_mou_6       3.937039
std_og_t2m_mou_7       3.859039
std_og_t2f_mou_6       3.937039
std_og_t2f_mou_7       3.859039
std_og_t2c_mou_6       3.937039
std_og_t2c_mou_7       3.859039
std_og_mou_6           3.937039
std_og_mou_7           3.859039
isd_og_mou_6           3.937039
isd_og_mou_7           3.859039
spl_og_mou_6           3.937039
spl_og_m

In [17]:
k3=tf[(tf>2) & (tf<4)].index

In [19]:
tf[(tf>0) & (tf<3.6)]

loc_og_t2o_mou          1.018010
std_og_t2o_mou          1.018010
loc_ic_t2o_mou          1.018010
last_date_of_month_7    0.601006
last_date_of_month_8    1.100011
last_date_of_month_9    1.659017
date_of_last_rech_6     1.607016
date_of_last_rech_7     1.767018
dtype: float64

In [20]:
col = ['loc_og_t2o_mou','std_og_t2o_mou','loc_ic_t2o_mou','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9']
for c in col:
    print(telco[c].value_counts())
    telco[c].fillna(telco[c].mode()[0], inplace=True)


0.0    98981
Name: loc_og_t2o_mou, dtype: int64
0.0    98981
Name: std_og_t2o_mou, dtype: int64
0.0    98981
Name: loc_ic_t2o_mou, dtype: int64
7/31/2014    99398
Name: last_date_of_month_7, dtype: int64
8/31/2014    98899
Name: last_date_of_month_8, dtype: int64
9/30/2014    98340
Name: last_date_of_month_9, dtype: int64


In [21]:
k=list(k)
k1=list(k1)
k2=list(k2)
k3=list(k3)
kf=k+k1+k2+k3

In [22]:
for col in kf:
    telco[col].fillna(0,inplace=True)

In [23]:
telco.drop(['night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8',
       'night_pck_user_9'],axis=1,inplace=True)

In [24]:
telco=telco[telco.columns[(telco.dtypes==float) | (telco.dtypes=='int64')]]

In [25]:
telco.isna().sum()

mobile_number         0
circle_id             0
loc_og_t2o_mou        0
std_og_t2o_mou        0
loc_ic_t2o_mou        0
arpu_6                0
arpu_7                0
arpu_8                0
arpu_9                0
onnet_mou_6           0
onnet_mou_7           0
onnet_mou_8           0
onnet_mou_9           0
offnet_mou_6          0
offnet_mou_7          0
offnet_mou_8          0
offnet_mou_9          0
roam_ic_mou_6         0
roam_ic_mou_7         0
roam_ic_mou_8         0
roam_ic_mou_9         0
roam_og_mou_6         0
roam_og_mou_7         0
roam_og_mou_8         0
roam_og_mou_9         0
loc_og_t2t_mou_6      0
loc_og_t2t_mou_7      0
loc_og_t2t_mou_8      0
loc_og_t2t_mou_9      0
loc_og_t2m_mou_6      0
loc_og_t2m_mou_7      0
loc_og_t2m_mou_8      0
loc_og_t2m_mou_9      0
loc_og_t2f_mou_6      0
loc_og_t2f_mou_7      0
loc_og_t2f_mou_8      0
loc_og_t2f_mou_9      0
loc_og_t2c_mou_6      0
loc_og_t2c_mou_7      0
loc_og_t2c_mou_8      0
loc_og_t2c_mou_9      0
loc_og_mou_6    

In [26]:
uni= telco.nunique().reset_index().rename(columns = {'index': 'feature', 0: 'nunique'})
print(uni[uni['nunique'] == 1])

              feature  nunique
1           circle_id        1
2      loc_og_t2o_mou        1
3      std_og_t2o_mou        1
4      loc_ic_t2o_mou        1
57   std_og_t2c_mou_6        1
58   std_og_t2c_mou_7        1
59   std_og_t2c_mou_8        1
60   std_og_t2c_mou_9        1
109  std_ic_t2o_mou_6        1
110  std_ic_t2o_mou_7        1
111  std_ic_t2o_mou_8        1
112  std_ic_t2o_mou_9        1


We can drop these columns as it wont add any value to the model

ARPU,MOU(onnet,offnet,T2c,AON

In [27]:
telco.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 210 columns):
 #   Column              Dtype  
---  ------              -----  
 0   mobile_number       int64  
 1   circle_id           int64  
 2   loc_og_t2o_mou      float64
 3   std_og_t2o_mou      float64
 4   loc_ic_t2o_mou      float64
 5   arpu_6              float64
 6   arpu_7              float64
 7   arpu_8              float64
 8   arpu_9              float64
 9   onnet_mou_6         float64
 10  onnet_mou_7         float64
 11  onnet_mou_8         float64
 12  onnet_mou_9         float64
 13  offnet_mou_6        float64
 14  offnet_mou_7        float64
 15  offnet_mou_8        float64
 16  offnet_mou_9        float64
 17  roam_ic_mou_6       float64
 18  roam_ic_mou_7       float64
 19  roam_ic_mou_8       float64
 20  roam_ic_mou_9       float64
 21  roam_og_mou_6       float64
 22  roam_og_mou_7       float64
 23  roam_og_mou_8       float64
 24  roam_og_mou_9       float64

### Identifying & filtering only the High Value Customer

In [ ]:
telco['HVC']=(telco['total_rech_amt_6']+telco['total_rech_amt_7'])/2.0

In [ ]:
telco['HVC']

0        307.0
1        229.0
2        241.5
3        270.0
4        273.0
         ...  
99994     85.0
99995    110.0
99996     59.5
99997    369.5
99998    355.0
Name: HVC, Length: 99999, dtype: float64

In [ ]:
np.percentile(telco['HVC'],70)

368.5

In [ ]:
telco['HVC']=telco['HVC'].apply(lambda x:0 if x<368.5 else 1)

In [ ]:
telco=telco[telco['HVC']==1]

In [ ]:
telco.shape

(30011, 211)

### Identifying the target variable

In [28]:
telco['churn']=np.where(((telco['total_ic_mou_9']==0)&(telco['total_og_mou_9']==0)&(telco['vol_2g_mb_9']==0)&(telco['vol_3g_mb_9']==0)),1,0)

In [29]:
telco[['churn','total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].head()

,churn,total_ic_mou_9,total_og_mou_9,vol_2g_mb_9,vol_3g_mb_9
0,1,0.00,0.00,0.0,0.00
1,0,188.04,72.11,0.0,0.00
2,0,517.74,124.94,0.0,8.42
3,0,305.38,362.54,0.0,0.00
4,0,205.31,104.59,0.0,0.00


### Dropping all columns related to 9th Month to avoid overfitting the model

In [30]:
telco.filter(like='_9',axis=1).columns

Index(['arpu_9', 'onnet_mou_9', 'offnet_mou_9', 'roam_ic_mou_9',
       'roam_og_mou_9', 'loc_og_t2t_mou_9', 'loc_og_t2m_mou_9',
       'loc_og_t2f_mou_9', 'loc_og_t2c_mou_9', 'loc_og_mou_9',
       'std_og_t2t_mou_9', 'std_og_t2m_mou_9', 'std_og_t2f_mou_9',
       'std_og_t2c_mou_9', 'std_og_mou_9', 'isd_og_mou_9', 'spl_og_mou_9',
       'og_others_9', 'total_og_mou_9', 'loc_ic_t2t_mou_9', 'loc_ic_t2m_mou_9',
       'loc_ic_t2f_mou_9', 'loc_ic_mou_9', 'std_ic_t2t_mou_9',
       'std_ic_t2m_mou_9', 'std_ic_t2f_mou_9', 'std_ic_t2o_mou_9',
       'std_ic_mou_9', 'total_ic_mou_9', 'spl_ic_mou_9', 'isd_ic_mou_9',
       'ic_others_9', 'total_rech_num_9', 'total_rech_amt_9', 'max_rech_amt_9',
       'last_day_rch_amt_9', 'total_rech_data_9', 'max_rech_data_9',
       'count_rech_2g_9', 'count_rech_3g_9', 'av_rech_amt_data_9',
       'vol_2g_mb_9', 'vol_3g_mb_9', 'arpu_3g_9', 'arpu_2g_9', 'monthly_2g_9',
       'sachet_2g_9', 'monthly_3g_9', 'sachet_3g_9', 'fb_user_9'],
      dtype='object')

In [31]:
telco.drop(telco.filter(like='_9',axis=1).columns,axis=1,inplace=True)

### Important Factors

Avg products being used per month,Net Incoming Calls(Incoming MOU-Outgoing MOU) per month,Avg recharges per month()

In [32]:
telco[['total_rech_num_6','count_rech_2g_6','count_rech_3g_6']]

,total_rech_num_6,count_rech_2g_6,count_rech_3g_6
0,4,0.0,1.0
1,4,0.0,0.0
2,5,0.0,0.0
3,10,0.0,0.0
4,5,1.0,0.0
...,...,...,...
99994,2,0.0,0.0
99995,5,0.0,0.0
99996,5,2.0,0.0
99997,3,1.0,2.0


In [33]:
telco.filter(like='rech',axis=1).head(15)

,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8
0,4,3,2,362,252,252,252,252,252,1.0,1.0,1.0,252.0,252.0,252.0,0.0,0.0,0.0,1.0,1.0,1.0,252.0,252.0,252.0
1,4,9,11,74,384,283,44,154,65,0.0,1.0,2.0,0.0,154.0,25.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,154.0,50.0
2,5,4,2,168,315,116,86,200,86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10,11,18,230,310,601,60,50,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,6,3,196,350,287,56,110,110,1.0,0.0,0.0,56.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0
5,2,2,3,120,0,130,120,0,130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,15,10,11,499,222,294,90,37,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,5,5,7,1580,790,3638,1580,790,1580,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,19,21,14,437,601,120,90,154,30,0.0,2.0,3.0,0.0,154.0,23.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,177.0,69.0
9,4,2,5,220,195,210,110,154,50,0.0,1.0,0.0,0.0,154.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,154.0,0.0


In [35]:
100*telco.isnull().sum()/len(telco) # Verfication

mobile_number         0.0
circle_id             0.0
loc_og_t2o_mou        0.0
std_og_t2o_mou        0.0
loc_ic_t2o_mou        0.0
arpu_6                0.0
arpu_7                0.0
arpu_8                0.0
onnet_mou_6           0.0
onnet_mou_7           0.0
onnet_mou_8           0.0
offnet_mou_6          0.0
offnet_mou_7          0.0
offnet_mou_8          0.0
roam_ic_mou_6         0.0
roam_ic_mou_7         0.0
roam_ic_mou_8         0.0
roam_og_mou_6         0.0
roam_og_mou_7         0.0
roam_og_mou_8         0.0
loc_og_t2t_mou_6      0.0
loc_og_t2t_mou_7      0.0
loc_og_t2t_mou_8      0.0
loc_og_t2m_mou_6      0.0
loc_og_t2m_mou_7      0.0
loc_og_t2m_mou_8      0.0
loc_og_t2f_mou_6      0.0
loc_og_t2f_mou_7      0.0
loc_og_t2f_mou_8      0.0
loc_og_t2c_mou_6      0.0
loc_og_t2c_mou_7      0.0
loc_og_t2c_mou_8      0.0
loc_og_mou_6          0.0
loc_og_mou_7          0.0
loc_og_mou_8          0.0
std_og_t2t_mou_6      0.0
std_og_t2t_mou_7      0.0
std_og_t2t_mou_8      0.0
std_og_t2m_m

In [36]:
telco.nunique()

mobile_number         99999
circle_id                 1
loc_og_t2o_mou            1
std_og_t2o_mou            1
loc_ic_t2o_mou            1
arpu_6                85681
arpu_7                85308
arpu_8                83615
onnet_mou_6           24313
onnet_mou_7           24336
onnet_mou_8           24089
offnet_mou_6          31140
offnet_mou_7          31023
offnet_mou_8          30908
roam_ic_mou_6          6512
roam_ic_mou_7          5230
roam_ic_mou_8          5315
roam_og_mou_6          8038
roam_og_mou_7          6639
roam_og_mou_8          6504
loc_og_t2t_mou_6      13539
loc_og_t2t_mou_7      13411
loc_og_t2t_mou_8      13336
loc_og_t2m_mou_6      20905
loc_og_t2m_mou_7      20637
loc_og_t2m_mou_8      20544
loc_og_t2f_mou_6       3860
loc_og_t2f_mou_7       3863
loc_og_t2f_mou_8       3807
loc_og_t2c_mou_6       2235
loc_og_t2c_mou_7       2426
loc_og_t2c_mou_8       2516
loc_og_mou_6          26372
loc_og_mou_7          26091
loc_og_mou_8          25990
std_og_t2t_mou_6    

### Data Preperation

In [ ]:
# Deriving   a categorical variable to analyse trend of  churning customers
# analysing outgoing  and data recharge trend if the customer is about to churn there will be -ve trend 
telco['Og_trend']=   lamda(x,y: if )